In [1]:
# get libs
import os
import requests
from bs4 import BeautifulSoup
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
# set url
url = "https://www.butterbeready.com/seafood-boil-with-garlic-butter-sauce/"
# url = "https://www.spendwithpennies.com/wprm_print/215236"
# url = "https://www.butterbeready.com/seafood-boil-with-garlic-butter-sauce/print/41509/"

In [14]:
# get texts
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    divs = soup.find_all('div', class_='tasty-recipes-entry-content')
    print(f"Status code: {response.status_code}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Status code: 200


In [15]:
# extract and print the text from each paragraph
texts_from_url = []

for p in paragraphs:
    texts_from_url.append(p.get_text())

for d in divs:
    texts_from_url.append(d.get_text())

In [16]:
# flatten
flatten_texts_from_url = ' '.join(texts_from_url)

In [17]:
flatten_texts_from_url

"This is the BEST Seafood Boil recipe you can whip up right at home, y’all. It’s loaded with jumbo shrimp, crab, andouille sausage, potatoes, eggs, and corn on the cob that’s simmered in a flavor-filled broth of cajun spices and aromatics. All that hearty goodness gets drowned and smothered in an epic garlic butter sauce. This finger-licking-good seafood boil recipe is an easy meal that serves a crowd! Mmmhmmm, seafood boils just cannot be beat, especially during these summer months. The delicious mess of seafood, sausage, potatoes, and corn all drenched in sauce and dripping from your fingertips…whew. The entire experience is so much fun and so dang good huh. Buckle up and let’s get into this recipe! This easy to make boil is brimming with all the meaty, hearty bites like andouille, shrimp, and crab. And then there’s filling elements like potatoes, corn, and hard-boiled eggs to amp up the fullness of it all. In my honest opinion, a seafood boil is questionable if it doesn’t include bi

In [18]:
# instantiate openai client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [19]:
# get recipe summarizer
my_assistant = client.beta.assistants.retrieve(os.getenv("OPENAI_ASST_ID"))
print(my_assistant)

Assistant(id='asst_1VheFuNiBaFWHIuP3gBQAkPp', created_at=1699510971, description=None, file_ids=[], instructions='You are a helpful recipe summarizer. The user will give you raw texts taken from a recipe website, and you will analyze the texts and succinctly extract the recipe information, including ingredients and steps, presenting them in a clear and concise format.\n\nBe sure in your response to include measurements of each ingredient, so that the user knows how much each ingredient is needed.\n\nIgnore non recipe related texts, such as comments, picture captions, etc.\n', metadata={}, model='gpt-4-1106-preview', name='Recipe Summarizer', object='assistant', tools=[])


In [20]:
# start thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_rJdjhW5CivjvFURBOYdLRwpx', created_at=1699516101, metadata={}, object='thread')


In [21]:
# send user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Raw recipe tex: ```{flatten_texts_from_url}```"
)

In [22]:
# run assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=my_assistant.id,
)

In [26]:
# check run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [27]:
# get response
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [29]:
# %% cell 13
print("Response from assistant: \n")
last_assistant_message = None
for message in messages.data:
    if message.role == "assistant":
        last_assistant_message = message

if last_assistant_message:
    print(f"Assistant: {last_assistant_message.content[0].text.value}")

Response from assistant: 

Assistant: ### Ingredients

#### For the Cajun Seafood Boil:
- 3 quarts water
- 1 (12-ounce) can of beer (optional but recommended)
- 3 tablespoons Cajun seasoning
- 1 tablespoon Old Bay seasoning
- Hot sauce, to taste
- 1 medium yellow onion, roughly sliced into half-moons
- 1 large lemon, cut into wedges (plus more for serving)
- 12 ounces andouille sausage, sliced into rounds
- 1 lb baby potatoes, red or gold (or a mix of both)
- 1 lb pre-cooked snow crab leg clusters, fresh or frozen
- 1 – 1 ½ lbs jumbo shrimp, shell-on or peeled
- 4–6 ears sweet corn on the cob (mini ones recommended)
- 4–6 hard-boiled eggs (optional)

#### For the Garlic Butter Sauce:
- 1 cup (2 sticks) unsalted butter
- 10 cloves of garlic, finely minced or pressed
- 2 tablespoons lemon juice, freshly squeezed
- 1 tablespoon Old Bay seasoning
- 1 tablespoon fresh chopped parsley
- 1 teaspoon Cajun seasoning
- 1 teaspoon smoked paprika
- Hot sauce, to taste

### Instructions

1. **Prepa